# DATA PROCESSING

This notebook is designed for data processing and serves as a proof of concept in creating a pipeline for varoius machine learning models. The data used is freely available on https://opendata.elia.be/pages/home/.

In parallel with this notebook, a package of simple functions is created to transform the data, called **transform_data**.

Since we have several datasets, we store them in a catalog of dataframes. We then iterate through the dataframes one by one and perform the following operations:
* renaming variables
* removing variables with missing values or no information value
* selecting a subset of the data
* adding indicators for holidays and working days
* converting time data to UTC format
* adding frequency to time data (and resolving missing values if necessary)

In [1]:
import os
from importlib import reload

import datetime
from workalendar.europe import Belgium

import pandas as pd
import transform_data
import feature_selection

In [177]:
reload(transform_data)
reload(feature_selection)

<module 'feature_selection' from 'd:\\Uni_FJFI\\NMS3\\DP\\project_I\\feature_selection.py'>

In [4]:
data_source = "../data/data_raw/"
data_pickle = "../data/data_pickle/"

## 1. Data exploration

In [3]:
data_files = {}

print("Data files:")
for i, filename in enumerate(os.listdir(data_source)):
    with open(os.path.join(data_source, filename), 'r') as file:
        data =  pd.read_csv(file, sep = ";", low_memory = False)
        data_files[filename[:-4]] = data
    print(f"{i+1}: {filename}")

Data files:
1: imbalance_prices_q.csv
2: photovoltaic_q.csv
3: total_load_q.csv
4: wind_q.csv


In [33]:
for k, v in data_files.items():
    data = v
    data.rename(columns=transform_data.name_mapper, inplace = True)
    data = transform_data.drop_nulls_and_redundants(data)
    data_files[k] = data.copy()

### 1.1 Imbalance prices

In [134]:
df = data_files["imbalance_prices_q"].copy()
df.head()

,datetime,net_regulation_volume,system_imbalance,alpha,marginal_incremental_price,marginal_decremental_price,positive_imbalance_price,negative_imbalance_price
0,2023-08-31T23:45:00+02:00,128.326,-141.174,0.0,109.64,64.64,109.64,109.64
1,2023-08-31T23:30:00+02:00,95.703,-106.571,0.0,111.67,64.64,111.67,111.67
2,2023-08-31T23:15:00+02:00,-28.812,43.858,0.0,109.64,57.56,57.56,57.56
3,2023-08-31T23:00:00+02:00,26.423,-43.184,0.0,125.31,64.54,125.31,125.31
4,2023-08-31T22:45:00+02:00,36.008,-33.795,0.0,109.64,64.64,109.64,109.64


In [142]:
# save "datetime" for later use
datetime_col = df.datetime.copy()

In [28]:
# is there any difference between positive and negative price
print("Max absolute difference:", max(abs(df.positive_imbalance_price - df.negative_imbalance_price)))

Max absolute difference: 0.0


In [35]:
df = df.rename(columns={"positive_imbalance_price": "imbalance_price"})
df = df.drop("negative_imbalance_price", axis=1)
df.head()

,datetime,net_regulation_volume,system_imbalance,alpha,marginal_incremental_price,marginal_decremental_price,imbalance_price
0,2023-08-31T23:45:00+02:00,128.326,-141.174,0.0,109.64,64.64,109.64
1,2023-08-31T23:30:00+02:00,95.703,-106.571,0.0,111.67,64.64,111.67
2,2023-08-31T23:15:00+02:00,-28.812,43.858,0.0,109.64,57.56,57.56
3,2023-08-31T23:00:00+02:00,26.423,-43.184,0.0,125.31,64.54,125.31
4,2023-08-31T22:45:00+02:00,36.008,-33.795,0.0,109.64,64.64,109.64


In [40]:
# save as pkl format - saves all timestamps formats and saves disk space
df.to_pickle(os.path.join(data_pickle, "imbalance_prices_q.pkl"))

### 1.2 Total load

In [190]:
df = data_files["total_load_q"].copy()
df.head()

,datetime,total_load,most_recent_forecast,most_recent_p10,most_recent_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast
0,2023-08-31T23:45:00+02:00,8072.71,8226.03,7909.66,8542.40,8189.81,7863.34,8516.28,8311.12
1,2023-08-31T23:30:00+02:00,8219.05,8353.07,8031.82,8674.33,8309.88,7978.63,8641.14,8459.30
2,2023-08-31T23:15:00+02:00,8341.76,8475.07,8149.12,8801.02,8439.19,8102.78,8775.60,8592.83
3,2023-08-31T23:00:00+02:00,8451.67,8621.29,8289.71,8952.86,8555.00,8213.97,8896.03,8713.50
4,2023-08-31T22:45:00+02:00,8710.15,8564.66,8245.01,8884.31,8611.00,8281.62,8940.37,8778.15


In [191]:
to_keep = [
    "datetime",
    "total_load",
    "most_recent_forecast",
    "most_recent_p10",
    "most_recent_p90",
    "day-ahead_6pm_forecast"
]

df = feature_selection.keep_columns(df, to_keep)
transform_data.add_suffix_to_columns(df, to_keep[2:], "_load")
df.head()

,datetime,total_load,most_recent_forecast_load,most_recent_p10_load,most_recent_p90_load,day-ahead_6pm_forecast_load
0,2023-08-31T23:45:00+02:00,8072.71,8226.03,7909.66,8542.40,8189.81
1,2023-08-31T23:30:00+02:00,8219.05,8353.07,8031.82,8674.33,8309.88
2,2023-08-31T23:15:00+02:00,8341.76,8475.07,8149.12,8801.02,8439.19
3,2023-08-31T23:00:00+02:00,8451.67,8621.29,8289.71,8952.86,8555.00
4,2023-08-31T22:45:00+02:00,8710.15,8564.66,8245.01,8884.31,8611.00


In [192]:
df.to_pickle(os.path.join(data_pickle, "total_load_q.pkl"))

### 1.3 Wind power

In [181]:
df = data_files["wind_q"].copy()
df.head()

,datetime,offshore/onshore,region,grid_connection_type,measured_&_upscaled,most_recent_forecast,most_recent_p10,most_recent_p90,day_ahead_11am_forecast,day_ahead_11am_p10,day_ahead_11am_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast,week-ahead_p10,week-ahead_p90,monitored_capacity,load_factor,decremental_bid_indicator
0,2023-08-31T23:45:00+02:00,Onshore,Flanders,Dso,37.94,88.2,45.5,142.6,143.6,71.9,232.9,136.1,72.3,177.8,27.9,6.6,95.5,1406.84,0.03,''
1,2023-08-31T23:45:00+02:00,Offshore,Federal,Elia,187.67,233.9,122.5,371.1,468.2,207.1,703.9,459.3,311.7,618.2,41.2,17.4,240.0,2262.10,0.08,''
2,2023-08-31T23:45:00+02:00,Onshore,Flanders,Elia,17.33,29.8,15.4,45.8,43.8,23.3,71.0,42.0,19.3,59.2,9.3,3.5,31.6,376.15,0.05,''
3,2023-08-31T23:45:00+02:00,Onshore,Wallonia,Dso,94.79,138.8,80.4,219.4,129.1,60.7,234.3,129.0,62.6,176.4,28.1,1.9,126.9,1093.35,0.09,''
4,2023-08-31T23:45:00+02:00,Onshore,Wallonia,Elia,6.42,17.8,10.3,43.9,14.1,5.9,45.4,13.9,8.8,29.4,1.3,0.6,22.6,151.10,0.04,''


In [182]:
vars_list = ["offshore/onshore", "region", "grid_connection_type"]
merger = ["datetime"]
cols = [col for col in df.columns if col not in vars_list+merger]

data_final = feature_selection.transform_columns_sum(df, cols, vars_list)
print("DF shape:", data_final.shape)
data_final.head()

DF shape: (58364, 17)


,measured_&_upscaled,most_recent_forecast,most_recent_p10,most_recent_p90,day_ahead_11am_forecast,day_ahead_11am_p10,day_ahead_11am_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast,week-ahead_p10,week-ahead_p90,monitored_capacity,load_factor,decremental_bid_indicator,datetime
0,344.15,508.5,274.1,822.8,798.8,368.9,1287.5,780.3,474.7,1061.0,107.8,30.0,516.6,5289.54,0.29,'''''''''',2023-08-31T23:45:00+02:00
1,405.96,499.6,272.2,808.8,782.3,372.9,1253.6,772.5,470.6,1063.0,107.5,29.9,516.7,5289.54,0.32,'''''''''',2023-08-31T23:30:00+02:00
2,453.12,490.3,270.6,794.2,762.4,392.6,1211.4,760.1,467.3,1070.9,107.5,29.4,516.1,5289.54,0.37,'''''''''',2023-08-31T23:15:00+02:00
3,414.12,476.3,268.3,771.7,742.1,413.6,1176.9,731.9,449.5,1067.4,105.6,28.3,514.0,5289.54,0.34,'''''''''',2023-08-31T23:00:00+02:00
4,359.41,458.2,261.7,740.6,721.9,422.3,1144.1,683.1,404.7,1032.1,102.3,27.7,511.1,5289.54,0.32,'''''''''',2023-08-31T22:45:00+02:00


In [183]:
to_keep = [
    "datetime",
    "measured_&_upscaled",
    "most_recent_forecast",
    "most_recent_p10",
    "most_recent_p90",
    "day-ahead_6pm_forecast",
]

data_final = feature_selection.keep_columns(data_final, to_keep)
transform_data.add_suffix_to_columns(data_final, to_keep[1:], "_wind")
data_final.head()

,measured_&_upscaled_wind,most_recent_forecast_wind,most_recent_p10_wind,most_recent_p90_wind,day-ahead_6pm_forecast_wind,datetime
0,344.15,508.5,274.1,822.8,780.3,2023-08-31T23:45:00+02:00
1,405.96,499.6,272.2,808.8,772.5,2023-08-31T23:30:00+02:00
2,453.12,490.3,270.6,794.2,760.1,2023-08-31T23:15:00+02:00
3,414.12,476.3,268.3,771.7,731.9,2023-08-31T23:00:00+02:00
4,359.41,458.2,261.7,740.6,683.1,2023-08-31T22:45:00+02:00


In [184]:
data_final.to_pickle(os.path.join(data_pickle, "wind_q.pkl"))

### 1.4 Photovoltaic power

In [185]:
df = data_files["photovoltaic_q"].copy()
df.head()

,datetime,region,measured_&_upscaled,most_recent_forecast,most_recent_p10,most_recent_p90,day_ahead_11am_forecast,day_ahead_11am_p10,day_ahead_11am_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast,week-ahead_p10,week-ahead_p90,monitored_capacity,load_factor
0,2023-08-31T23:45:00+02:00,Namur,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.799,NaN
1,2023-08-31T23:45:00+02:00,Flemish-Brabant,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,688.854,NaN
2,2023-08-31T23:45:00+02:00,Flanders,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5613.795,NaN
3,2023-08-31T23:45:00+02:00,Brussels,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254.665,NaN
4,2023-08-31T23:45:00+02:00,East-Flanders,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1471.961,NaN


In [186]:
# in "measured_&_upscaled" we replace NaN with 0
df["measured_&_upscaled"] = df["measured_&_upscaled"].fillna(0)
df.head()

,datetime,region,measured_&_upscaled,most_recent_forecast,most_recent_p10,most_recent_p90,day_ahead_11am_forecast,day_ahead_11am_p10,day_ahead_11am_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast,week-ahead_p10,week-ahead_p90,monitored_capacity,load_factor
0,2023-08-31T23:45:00+02:00,Namur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.799,NaN
1,2023-08-31T23:45:00+02:00,Flemish-Brabant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,688.854,NaN
2,2023-08-31T23:45:00+02:00,Flanders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5613.795,NaN
3,2023-08-31T23:45:00+02:00,Brussels,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254.665,NaN
4,2023-08-31T23:45:00+02:00,East-Flanders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1471.961,NaN


In [187]:
vars_list = ["region"]
merger = ["datetime"]
cols = [col for col in df.columns if col not in vars_list+merger]

data_final = feature_selection.transform_columns_sum(df, cols, vars_list)
print("DF shape:", data_final.shape)
data_final.head()

DF shape: (58364, 16)


,measured_&_upscaled,most_recent_forecast,most_recent_p10,most_recent_p90,day_ahead_11am_forecast,day_ahead_11am_p10,day_ahead_11am_p90,day-ahead_6pm_forecast,day-ahead_6pm_p10,day-ahead_6pm_p90,week-ahead_forecast,week-ahead_p10,week-ahead_p90,monitored_capacity,load_factor,datetime
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23032.052,NaN,2023-08-31T23:45:00+02:00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23032.052,0.0,2023-08-31T23:30:00+02:00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23032.052,0.0,2023-08-31T23:15:00+02:00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23032.052,0.0,2023-08-31T23:00:00+02:00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23032.052,0.0,2023-08-31T22:45:00+02:00


In [188]:
to_keep = [
    "datetime",
    "measured_&_upscaled",
    "most_recent_forecast",
    "most_recent_p10",
    "most_recent_p90",
    "day-ahead_6pm_forecast",
]

data_final = feature_selection.keep_columns(data_final, to_keep)
transform_data.add_suffix_to_columns(data_final, to_keep[1:], "_solar")
data_final.head()

,measured_&_upscaled_solar,most_recent_forecast_solar,most_recent_p10_solar,most_recent_p90_solar,day-ahead_6pm_forecast_solar,datetime
0,0.0,0.0,0.0,0.0,0.0,2023-08-31T23:45:00+02:00
1,0.0,0.0,0.0,0.0,0.0,2023-08-31T23:30:00+02:00
2,0.0,0.0,0.0,0.0,0.0,2023-08-31T23:15:00+02:00
3,0.0,0.0,0.0,0.0,0.0,2023-08-31T23:00:00+02:00
4,0.0,0.0,0.0,0.0,0.0,2023-08-31T22:45:00+02:00


In [189]:
data_final.to_pickle(os.path.join(data_pickle, "photovoltaic_q.pkl"))

### 1.5 Calendar info + UTC datetime

In [157]:
cal = Belgium()

start_date = datetime.datetime.strptime(datetime_col[len(datetime_col)-1], '%Y-%m-%dT%H:%M:%S%z').date()
end_date = datetime.datetime.strptime(datetime_col[0], '%Y-%m-%dT%H:%M:%S%z').date()
period = [start_date, end_date]

is_be_holiday = transform_data.add_holidays(datetime_col, "BE", [y for y in range(start_date.year, end_date.year+1)])
is_be_not_working = transform_data.add_not_working_days(datetime_col, cal, period)

In [174]:
df_time = datetime_col.to_frame().assign(holiday = is_be_holiday.astype(int), not_working = is_be_not_working.astype(int), datetime_utc = datetime_col)
df_time.head()

,datetime,holiday,not_working,datetime_utc
0,2023-08-31T23:45:00+02:00,0,0,2023-08-31T23:45:00+02:00
1,2023-08-31T23:30:00+02:00,0,0,2023-08-31T23:30:00+02:00
2,2023-08-31T23:15:00+02:00,0,0,2023-08-31T23:15:00+02:00
3,2023-08-31T23:00:00+02:00,0,0,2023-08-31T23:00:00+02:00
4,2023-08-31T22:45:00+02:00,0,0,2023-08-31T22:45:00+02:00


In [175]:
columns = ["datetime_utc"]

transform_data.columns_to_utc(df_time, columns, '%Y-%m-%dT%H:%M:%S%z')
## QUESTION: While utc=False => does not work due to different time shift: time change => summer time vs winter time?
df_time.head()

,datetime,holiday,not_working,datetime_utc
0,2023-08-31T23:45:00+02:00,0,0,2023-08-31 21:45:00
1,2023-08-31T23:30:00+02:00,0,0,2023-08-31 21:30:00
2,2023-08-31T23:15:00+02:00,0,0,2023-08-31 21:15:00
3,2023-08-31T23:00:00+02:00,0,0,2023-08-31 21:00:00
4,2023-08-31T22:45:00+02:00,0,0,2023-08-31 20:45:00


In [176]:
df_time.to_pickle(os.path.join(data_pickle, "calendar_info.pkl"))